### 오늘의 연습
### 최소 20개 이상의 자료에 대한 좌표를 지정.
### 또한 각 10개 이상의 동일 그룹에 대해 0, 1, ~ 등으로 출력 레이블을 지정
### 테스트 좌표는 4개 이상 지정

In [ ]:
#제 1, 2, 3, 4사분면 나누기
import numpy as np
import random
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #입력 크기, 히든 레이어 크기, 출력 크기, 실수값으로 조지기
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #입력x히든 사이즈로 난수 배열 만들고 실수로 값 변환
        self.params['b1'] = np.zeros(hidden_size)   #히든 레이어 개수만큼 
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0]) #np.sum(y==t) y==t인 case의 횟수를 저장, 이를 평균을 내줌
        return accuracy                               #그렇게 구한 정확도를 반환
    
    def numerical_gradient(self, x, t):                 #손실함수를 w에 대해 미분함 -> gradient를 반환
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):   #x=20x2, t=20x1
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]  #20
        #forward
        a1 = np.dot(x, W1) + b1 
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads

#학습데이터 20개 이상 생성: 100개
train_x = np.random.uniform(-5, 5, (100, 2))
train_x = np.round(train_x, decimals=2)

# 출력 레이블: 1, 2, 3, 4
train_t = np.where((train_x[:, 0] > 0) & (train_x[:, 1] > 0), 1, 
    np.where((train_x[:, 0] < 0) & (train_x[:, 1] > 0), 2,
    np.where((train_x[:, 0] < 0) & (train_x[:, 1] < 0), 3,
    np.where((train_x[:, 0] > 0) & (train_x[:, 1] < 0), 4,  
    0)))).reshape(-1, 1)

# 테스트 데이터 4개 이상: 20
test_x = np.random.uniform(-5, 5, (20, 2))
test_x = np.round(test_x, decimals=2)

# x_test의 예상 레이블 4개
test_t = np.where((test_x[:, 0] > 0) & (test_x[:, 1] > 0), 1, 
    np.where((test_x[:, 0] < 0) & (test_x[:, 1] > 0), 2,
    np.where((test_x[:, 0] < 0) & (test_x[:, 1] < 0), 3,
    np.where((test_x[:, 0] > 0) & (test_x[:, 1] < 0), 4,  
    0)))).reshape(-1, 1)

network = TwoLayerNet(2, 3, 1) #(784, 50, 10)-> (2, 5||3, 1), (#입력, #히든 레이어, #출력)
iters_num = 10000                 #10000->100
train_size = train_x.shape[0] #20
learning_rate = 0.01
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = 10
for i in range(iters_num):
    grad = network.gradient(train_x, train_t)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(train_x, train_t)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(train_x, train_t)
        test_acc = network.accuracy(test_x, test_t)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        #print(i, "train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

print("[x_train, t_train]")
for j in range(100):
    print(f"{train_x[j], train_t[j]}")

print("\n[x_test, t_test]")
for k in range(20):
    print(f"{test_x[k], test_t[k]}")

[x_train, t_train]
(array([-4.1 , -4.85]), array([3]))
(array([-0.94, -0.75]), array([3]))
(array([4.48, 4.46]), array([1]))
(array([-0.44,  2.85]), array([2]))
(array([-1.52,  1.9 ]), array([2]))
(array([-3.77,  1.81]), array([2]))
(array([-3.86,  3.45]), array([2]))
(array([-4.01, -1.  ]), array([3]))
(array([-2.43, -3.71]), array([3]))
(array([-2.16, -4.75]), array([3]))
(array([-0.28, -3.53]), array([3]))
(array([ 1.79, -4.46]), array([4]))
(array([-2.85,  2.15]), array([2]))
(array([3.77, 1.42]), array([1]))
(array([-4.64, -4.69]), array([3]))
(array([ 0.89, -3.23]), array([4]))
(array([0.45, 4.15]), array([1]))
(array([3.96, 2.03]), array([1]))
(array([ 0.29, -2.6 ]), array([4]))
(array([-2.34, -0.11]), array([3]))
(array([-1.22,  3.38]), array([2]))
(array([-2.29, -4.05]), array([3]))
(array([ 1.65, -2.71]), array([4]))
(array([-4.45,  3.58]), array([2]))
(array([-0.03, -4.38]), array([3]))
(array([4.18, 0.68]), array([1]))
(array([-1.38,  1.12]), array([2]))
(array([-3.75, -0.8

In [1]:
#제 1, 2, 3, 4사분면 추정 시 정확도와 loss 측정
import numpy as np
import random
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #입력 크기, 히든 레이어 크기, 출력 크기, 실수값으로 조지기
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #입력x히든 사이즈로 난수 배열 만들고 실수로 값 변환
        self.params['b1'] = np.zeros(hidden_size)   #히든 레이어 개수만큼 
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0]) #np.sum(y==t) y==t인 case의 횟수를 저장, 이를 평균을 내줌
        return accuracy                               #그렇게 구한 정확도를 반환
    
    def numerical_gradient(self, x, t):                 #손실함수를 w에 대해 미분함 -> gradient를 반환
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):   #x=20x2, t=20x1
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]  #20
        #forward
        a1 = np.dot(x, W1) + b1 
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads

def one_hot(label, num_classes=4):
    one_hot = np.zeros((len(label), num_classes))
    for i, label in enumerate(label):
        one_hot[i, label-1] = 1
    return one_hot

#학습데이터 20개 이상 생성: 100개
train_x = np.random.uniform(-5, 5, (100, 2))
train_x = np.round(train_x, decimals=2)

# 출력 레이블: 1, 2, 3, 4
train_t = np.where((train_x[:, 0] > 0) & (train_x[:, 1] > 0), 1, 
    np.where((train_x[:, 0] < 0) & (train_x[:, 1] > 0), 2,
    np.where((train_x[:, 0] < 0) & (train_x[:, 1] < 0), 3,
    np.where((train_x[:, 0] > 0) & (train_x[:, 1] < 0), 4,  
    0)))).reshape(-1, 1)
train_t = one_hot(train_t)

# 테스트 데이터 4개 이상: 20
test_x = np.random.uniform(-5, 5, (20, 2))
test_x = np.round(test_x, decimals=2)

# x_test의 예상 레이블 4개
test_t = np.where((test_x[:, 0] > 0) & (test_x[:, 1] > 0), 1, 
    np.where((test_x[:, 0] < 0) & (test_x[:, 1] > 0), 2,
    np.where((test_x[:, 0] < 0) & (test_x[:, 1] < 0), 3,
    np.where((test_x[:, 0] > 0) & (test_x[:, 1] < 0), 4,  
    0)))).reshape(-1, 1)
test_t = one_hot(test_t)

network = TwoLayerNet(2, 100, 4) #(784, 50, 10)-> (2, 5||3, 1), (#입력, #히든 레이어, #출력)
iters_num = 10000                 #10000->100
train_size = train_x.shape[0] #20
learning_rate = 0.01
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = 10
for i in range(iters_num):
    grad = network.gradient(train_x, train_t)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(train_x, train_t)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(train_x, train_t)
        test_acc = network.accuracy(test_x, test_t)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(i, "train acc:" + str(train_acc) + ", test acc " + str(test_acc) + ", loss: " + str(loss))


0 train acc:0.28, test acc 0.35, loss: 1.387962088058786
10 train acc:0.28, test acc 0.35, loss: 1.3789820382115505
20 train acc:0.28, test acc 0.35, loss: 1.376181094020486
30 train acc:0.28, test acc 0.35, loss: 1.3750776991610314
40 train acc:0.28, test acc 0.35, loss: 1.3744719692579508
50 train acc:0.28, test acc 0.35, loss: 1.3740149401673005
60 train acc:0.28, test acc 0.35, loss: 1.3735993267517792
70 train acc:0.28, test acc 0.35, loss: 1.3731904285844394
80 train acc:0.28, test acc 0.35, loss: 1.372776122527186
90 train acc:0.28, test acc 0.35, loss: 1.3723514536939632
100 train acc:0.28, test acc 0.35, loss: 1.3719137565423905
110 train acc:0.28, test acc 0.35, loss: 1.371461080506537
120 train acc:0.28, test acc 0.35, loss: 1.370991673695684
130 train acc:0.28, test acc 0.35, loss: 1.3705038098134887
140 train acc:0.28, test acc 0.35, loss: 1.369995727325214
150 train acc:0.28, test acc 0.35, loss: 1.3694656054908447
160 train acc:0.28, test acc 0.35, loss: 1.36891155249730